# Analyze Price Data

In [2]:
import pandas as pd

# Read Data

In [47]:
dataparse = lambda dates: pd.datetime.strptime(dates, '%m-%Y')
file_name = 'monthly_data.xlsx'
dfs = pd.read_excel(file_name, sheet_name=None, 
                    parse_dates=["Fecha"],
                   date_parser=dataparse)

In [48]:
for key, value in dfs.items():
    dfs[key] = dfs[key].set_index("Fecha")

In [64]:
ts_dict = {key: dfs[key]["Último"] for key in dfs.keys()}

In [66]:
ts_dict["Cebada"]

Fecha
2006-01-01    144.800000
2006-02-01    141.500000
2006-03-01    135.750000
2006-04-01    128.500000
2006-05-01    119.250000
2006-06-01    116.666667
2006-07-01    113.600000
2006-08-01    117.000000
2006-09-01    125.750000
2006-10-01    139.800000
2006-11-01    153.500000
2006-12-01    154.000000
2007-01-01    153.000000
2007-02-01    155.250000
2007-03-01    154.750000
2007-04-01    157.000000
2007-05-01    179.000000
2007-06-01    168.500000
2007-07-01    159.600000
2007-08-01    181.000000
2007-09-01    209.500000
2007-10-01    222.200000
2007-11-01    207.000000
2007-12-01    222.800000
2008-01-01    237.500000
2008-02-01    222.500000
2008-03-01    219.200000
2008-04-01    209.000000
2008-05-01    183.250000
2008-06-01    196.400000
                 ...    
2016-06-01    158.000000
2016-07-01    145.000000
2016-08-01    143.600000
2016-09-01    141.250000
2016-10-01    141.000000
2016-11-01    144.250000
2016-12-01    148.000000
2017-01-01    147.400000
2017-02-01    147.7